In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from trl import setup_chat_format
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          pipeline, 
                          logging)
from sklearn.metrics import (accuracy_score, 
                             classification_report, 
                             confusion_matrix,
                             f1_score)
from sklearn.model_selection import train_test_split

In [2]:
print(f"pytorch version {torch.__version__}")

pytorch version 2.2.0+cu121


In [3]:
# get working directory
cwd = os.getcwd()
data_dir = os.path.join(cwd, 'data')
model_dir = os.path.join(cwd, 'model')

# load data and pre-process datasets
train_df = pd.read_csv(os.path.join(data_dir, 'gptTestNames.csv'))
# test_df = pd.read_csv(os.path.join(data_dir, 'gptTestNames.csv'))
# val_df = pd.read_csv(os.path.join(data_dir, 'gptValNames.csv'))

In [21]:
# X_train = list()
# X_test = list()
# for race in ["API", "White", "Black", "Hispanic"]:
#     train, test  = train_test_split(train_df[train_df.label==race], 
#                                     train_size=300,
#                                     test_size=300, 
#                                     random_state=42)
#     X_train.append(train)
#     X_test.append(test)

X_train, X_test  = train_test_split(train_df, 
                                train_size=int(len(train_df) * 0.0008),
                                test_size=int(len(train_df) * 0.0002), 
                                random_state=42)

# X_train = pd.concat(X_train).sample(frac=1, random_state=10)
# X_test = pd.concat(X_test)

# eval_idx = [idx for idx in train_df.index if idx not in list(train.index) + list(test.index)]
# X_eval = train_df[train_df.index.isin(eval_idx)]
X_train, X_eval = train_test_split(X_train, 
                                test_size=0.1,
                                random_state=42)
X_train = X_train.reset_index(drop=True)

In [5]:
len(X_train), len(X_test), len(X_eval)

(1867, 518, 208)

In [22]:
def generate_prompt(data_point):
    return f"""
            Category the name enclosed in square brackets into 1 of the following 4 categories: Asian Pacific Islander, Black, Hispanic, or White. 
            Your answer should only be the category name. 
            [{data_point["name"]}].
            ANSWER: {data_point["label"]}
            """.strip()

def generate_test_prompt(data_point):
    return f"""
            Guess the race of the name enclosed in square brackets into 1 of the following 4 categories: Asian, Black, Hispanic, or White. 
            Your answer should only be the category name.
            [{data_point["name"]}]
            ANSWER: """.strip()

X_train = pd.DataFrame(X_train.apply(generate_prompt, axis=1), 
                       columns=["name"])
X_eval = pd.DataFrame(X_eval.apply(generate_prompt, axis=1), 
                      columns=["name"])

y_true = X_test.label
X_test = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["name"])

train_data = Dataset.from_pandas(X_train)
eval_data = Dataset.from_pandas(X_eval)

In [18]:
def evaluate(y_true, y_pred):
    labels = ['API', 'Black', 'Hispanic', 'White']
    mapping = {'API': 0, 'Black': 1, 'Hispanic':2, 'White': 3}
    # def map_func(x):
    #     return mapping.get(x, 1)
    
    # y_true = np.vectorize(map_func)(y_true)
    # y_pred = np.vectorize(map_func)(y_pred)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')
    
    # Generate accuracy report
    # unique_labels = set(y_true)  # Get unique labels
    
    # for label in unique_labels:
    #     label_indices = [i for i in range(len(y_true)) 
    #                      if y_true[i] == label]
    #     label_y_true = [y_true[i] for i in label_indices]
    #     label_y_pred = [y_pred[i] for i in label_indices]
    #     accuracy = accuracy_score(label_y_true, label_y_pred)
    #     print(f'Accuracy for label {label}: {accuracy:.3f}')
        
    # Generate classification report
    class_report = classification_report(y_true=y_true, y_pred=y_pred, target_names=labels)
    print('\nClassification Report:')
    print(class_report)
    
    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred)#, labels=labels)
    print('\nConfusion Matrix:')
    print(conf_matrix)

In [7]:
model_name = "meta-llama/Llama-2-7b-chat-hf"

compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_quant_type="nf4", 
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=compute_dtype,
    quantization_config=bnb_config, 
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name, 
                                          trust_remote_code=True,
                                         )
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model, tokenizer = setup_chat_format(model, tokenizer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [24]:
def predict(test, model, tokenizer):
    y_pred = []
    for i in tqdm(range(len(test))):
    # for i in [69, 222, 676, 1270, 2060, 3684, 3827, 4472, 4799, 4972, 5120]:
        prompt = test.iloc[i]["name"]
        pipe = pipeline(task="text-generation", 
                        model=model, 
                        tokenizer=tokenizer, 
                        max_new_tokens = 4, 
                        # temperature = 0.01,
                        do_sample = False,
                       )
        result = pipe(prompt)
        answer = result[0]['generated_text'].split(":")[-1].lower()
        # print(prompt, answer)
        if "asian" in answer:
            y_pred.append("API")
        elif "black" in answer:
            y_pred.append("Black")
        elif "hispanic" in answer:
            y_pred.append("Hispanic")
        elif "white" in answer:
            y_pred.append("White")
        else:
            y_pred.append("none")
            print(prompt,answer)
    return y_pred

In [25]:
y_pred = predict(X_test, model, tokenizer)
# y_pred1 = predict(X_test, model, tokenizer)

  0%|          | 0/518 [00:00<?, ?it/s]/WAVE/projects/newsq_scu/xiaoxiao_git/Llama-race/venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/WAVE/projects/newsq_scu/xiaoxiao_git/Llama-race/venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
100%|██████████| 518/518 [07:34<00:00,  1.14it/s]


In [12]:
unique_res = set(y_pred)  # Get unique labels
print(unique_res)

unique_labels = set(y_true)
print(unique_labels)

labels = list(set(list(set(y_true))+list(set(y_pred))))
labels

{'Black', 'Hispanic', 'White', 'API'}
{'White', 'Hispanic', 'API', 'Black'}


['White', 'Hispanic', 'Black', 'API']

In [26]:
# y_pred = ['API' if 'Asia Pacific Islander' in x else x for x in y_pred]
evaluate(y_true, y_pred)

Accuracy: 0.309

Classification Report:
              precision    recall  f1-score   support

         API       0.03      1.00      0.07        11
       Black       0.37      0.24      0.29        80
    Hispanic       0.93      0.80      0.86        92
       White       0.88      0.17      0.28       335

    accuracy                           0.31       518
   macro avg       0.55      0.55      0.37       518
weighted avg       0.79      0.31      0.38       518


Confusion Matrix:
[[ 11   0   0   0]
 [ 50  19   3   8]
 [ 18   0  74   0]
 [244  32   3  56]]


In [15]:
conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=labels)

In [16]:
print(conf_matrix)

[[ 378    2    3  491]
 [   2    9    6  689]
 [  41    8   24 3431]
 [   3    0    0  101]]


In [27]:
f1_micro = f1_score(y_true, y_pred, average='micro')
f1_macro = f1_score(y_true, y_pred, average='macro')
f1_weighted = f1_score(y_true, y_pred, average='weighted')
print(f"F1 Score (Micro): {f1_micro:.3f}")
print(f"F1 Score (Macro): {f1_macro:.3f}")
print(f"F1 Score (Weighted): {f1_weighted:.3f}")

F1 Score (Micro): 0.309
F1 Score (Macro): 0.374
F1 Score (Weighted): 0.381


In [57]:
print(type(X_test))

<class 'pandas.core.frame.DataFrame'>
